In [ ]:
from pathlib import Path
import os
from raman_id.config import RAW_DIR, INTERIM_DIR
import json
import numpy as np
import h5py

In [ ]:
RAW_EXCELLENT_UNORIENTED_FOLDER = RAW_DIR / 'rruff_excellent_unoriented'
INTERIM_EXCELLENT_UNORIENTED_FOLDER = INTERIM_DIR / 'excellent_unoriented'
print(RAW_EXCELLENT_UNORIENTED_FOLDER)

def load_rruff_file(path: Path):
    with path.open('r') as f:
        return path.name, f.readlines()

In [ ]:
pathlist = RAW_EXCELLENT_UNORIENTED_FOLDER.rglob('Abellaite*.txt')
spectra=[]
for path in pathlist:
    print(path)
    spectrum_name, spectrum = load_rruff_file(path)
    spectra.append({"filename": spectrum_name, "spectrum": spectrum})

print(spectra[0])

In [ ]:
def parse_spectra(raw_spectrum_record):
    metadata= {'filename':raw_spectrum_record['filename']}
    raman_shifts=[]
    intensities=[]
    for line in raw_spectrum_record['spectrum']:
        if line.startswith('#'):
            metadata[line.split('=')[0].strip("#").replace(' ','_')]=line.split('=')[1].strip('\n')
        else:
            parsed_values = line.strip('\n').replace(' ','').split(',')
            if parsed_values !=['']:
                raman_shifts.append(float(parsed_values[0]))
                intensities.append(float(parsed_values[1]))
    spectral_data=np.array([raman_shifts,intensities])
    return {'metadata': metadata, 'spectral_data': spectral_data}
parse_spectra(spectra[0])


In [ ]:
pathlist = RAW_EXCELLENT_UNORIENTED_FOLDER.rglob('*.txt')
for path in pathlist:
    try:
        spectrum_name, spectrum = load_rruff_file(path)
        parsed_spectrum = parse_spectra({"filename": spectrum_name, "spectrum": spectrum})
        with h5py.File(f'{INTERIM_EXCELLENT_UNORIENTED_FOLDER}/{spectrum_name.replace('.txt','')}.h5','w') as h5:
            grp = h5.create_group('spectra')
            s = grp.create_group(spectrum_name.replace('.txt',''))
            s.create_dataset('data', data = parsed_spectrum['spectral_data'])
            for key, value in parsed_spectrum['metadata'].items():
                s.attrs[key] = value
    except:
        print(path)

In [ ]:
spectra[0]
with open('./test.json','w') as f:
    json.dump(spectra[0],f, indent=2)